In [31]:
import pandas as pd
import random

NUM_FILAS = 500
OUTPUT_FILE = "solicitudes.csv"

TIPOS = ["peticion", "queja", "reclamo", "felicitacion"]
#igual peso en todas las clases
TIPOS_PESOS = [0.25, 0.25, 0.25, 0.25]

# Inicio de frase (ruido + variación)
INICIO = [
" !!!El ciudadano REPORTA ",
"Se INFOrma sobre--- ",
" ***La comunidad__indica ",
"El usuario MANIFIESTA",
" HABITANTE del sector dice: ",
">>Se presenta una solicitud "
]

#frases generales
COMPLEMENTO = [
"mal estado de la vía principal!!",
" retraso en la recolección de basuras....",
"FALTA de iluminación pública ",
"ruido EXCESIVO!!! en horas de la noche",
"obras inconclusas que afectan la movilidad",
"árbol a punto de caer ",
"demora en atención médica!!!!",
"robos recientes en el barrio ",
"transporte público en MAL estado ",
]

# Frases típicas según tipo -> esto ayuda MUCHO a la clasificación
DETAILS_BY_TIPO = {
"peticion": [
"solicita respetuosamente se realice mantenimiento de la vía",
"pide que se incremente la frecuencia de recolección de basuras",
"requiere la instalación de nuevas luminarias en la zona",
"solicita mayor presencia de la policía en el barrio",
"pide la reparación urgente del parque infantil"
],
"queja": [
"presenta queja por el mal servicio de atención al usuario",
"manifiesta inconformidad con el ruido constante del establecimiento",
"se queja por la demora excesiva en los trámites",
"expone su molestia por la falta de respuesta a solicitudes anteriores",
"reclama por el trato descortés recibido por parte del funcionario"
],
"reclamo": [
"presenta reclamo formal por cobro incorrecto en la factura",
"exige revisión del valor facturado por el servicio de acueducto",
"radica reclamo por suspensión injustificada del servicio",
"pide la devolución de dinero cobrado en exceso",
"reclama por daños causados durante la ejecución de una obra"
],
"felicitacion": [
"expresa su agradecimiento por la rápida solución brindada",
"felicita al equipo de atención por su excelente servicio",
"reconoce la labor del personal que atendió la solicitud",
"destaca la organización del evento comunitario realizado",
"agradece la pronta respuesta a la petición presentada"
]
}

rows = []

for i in range(NUM_FILAS):

    tipo = random.choices(TIPOS, weights=TIPOS_PESOS, k=1)[0]

    fragmento = random.choice(INICIO)


    detalle_generico = random.choice(COMPLEMENTO)
    detalle_tipo = random.choice(DETAILS_BY_TIPO[tipo])

    texto = f"{fragmento} {detalle_generico} {detalle_tipo} [caso-{i+1}]"

    #agrega valores nan
    if random.random() < 0.05: # ~5% de textos nulos
        texto = None

    rows.append(
        {
            "texto": texto,
            "tipo": tipo
        }
    )

df = pd.DataFrame(rows)
df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

print(f"Archivo generado: {OUTPUT_FILE}")
print(df["tipo"].value_counts())
print(df.head())




Archivo generado: solicitudes.csv
tipo
queja           141
reclamo         134
peticion        121
felicitacion    104
Name: count, dtype: int64
                                               texto          tipo
0   ***La comunidad__indica  FALTA de iluminación...       reclamo
1  >>Se presenta una solicitud  robos recientes e...         queja
2   ***La comunidad__indica  transporte público e...         queja
3  El usuario MANIFIESTA mal estado de la vía pri...  felicitacion
4   ***La comunidad__indica  árbol a punto de cae...  felicitacion


In [43]:
import pandas as pd
import re


INPUT_FILE = "solicitudes.csv"
OUTPUT_FILE = "solicitudes_limpias.csv"

df = pd.read_csv(INPUT_FILE)


TYPES = ["peticion", "queja", "reclamo", "felicitacion"]

# -----------------------------
# 3. Stopwords (lista simple)
# -----------------------------
spanish_stopwords = {
"de","la","que","el","en","y","a","los","del","se","las","por","un","para",
"con","no","una","su","al","lo","como","mas","más","pero","sus","le","ya",
"o","este","sí","si","porque","esta","entre","cuando","muy","sin","sobre",
"tambien","también","me","hasta","hay","donde","quien","quién","desde",
"todo","nos","durante","todos","uno","les","ni","contra","otros","ese",
"eso","ante","ellos","e","esto","antes","algunos","unos","yo","otro","otras",
"otra","él","tanto","esa","estos","mucho","nada","muchos","cual","poco",
"cuales","ella","estar","estas","estás","esta","está","estamos","estais","estáis","estan","están","estar","estado"
}

# -----------------------------
# 4. Función de limpieza texto
# -----------------------------
def clean_text(text: str) -> str:
    if pd.isna(text):
        return ""

    # minúsculas
    text = text.lower()

    # quitar tag [caso-123]
    text = re.sub(r"\[caso-\d+\]", " ", text)

    # dejar solo letras, números y espacios (incluye acentos y ñ)
    text = re.sub(r"[^a-záéíóúñü0-9\s]", " ", text)

    # normalizar espacios
    text = re.sub(r"\s+", " ", text).strip()

    # eliminar stopwords
    tokens = [w for w in text.split() if w not in spanish_stopwords]

    return " ".join(tokens)


df["tipo"] = (
df["tipo"]
.astype(str)
.str.lower()
.str.strip()
)


df = df[df["tipo"].isin(TYPES)]


#Eliminar filas sin texto y limpiar
df = df.dropna(subset=["texto"])
df["texto_limpio"] = df["texto"].apply(clean_text)

# Quitar filas que queden vacías
df = df[df["texto_limpio"].str.len() > 0]


# Guarda
df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
print(f"Archivo limpio generado: {OUTPUT_FILE}")
print(df.head())

Archivo limpio generado: solicitudes_limpias.csv
                                               texto          tipo  \
0   ***La comunidad__indica  FALTA de iluminación...       reclamo   
1  >>Se presenta una solicitud  robos recientes e...         queja   
2   ***La comunidad__indica  transporte público e...         queja   
3  El usuario MANIFIESTA mal estado de la vía pri...  felicitacion   
4   ***La comunidad__indica  árbol a punto de cae...  felicitacion   

                                        texto_limpio  
0  comunidad indica falta iluminación pública rad...  
1  presenta solicitud robos recientes barrio quej...  
2  comunidad indica transporte público mal reclam...  
3  usuario manifiesta mal vía principal expresa a...  
4  comunidad indica árbol punto caer reconoce lab...  


In [44]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score
)

# 2. Carga data set limpio
df = pd.read_csv("solicitudes_limpias.csv")

print("Shape:", df.shape)
df.head()

Shape: (472, 3)


,texto,tipo,texto_limpio
0,***La comunidad__indica FALTA de iluminación...,reclamo,comunidad indica falta iluminación pública rad...
1,>>Se presenta una solicitud robos recientes e...,queja,presenta solicitud robos recientes barrio quej...
2,***La comunidad__indica transporte público e...,queja,comunidad indica transporte público mal reclam...
3,El usuario MANIFIESTA mal estado de la vía pri...,felicitacion,usuario manifiesta mal vía principal expresa a...
4,***La comunidad__indica árbol a punto de cae...,felicitacion,comunidad indica árbol punto caer reconoce lab...


In [45]:
#  Selecciona las columnas
X = df["texto_limpio"].astype(str)
y = df["tipo"].astype(str)

# muestra distribucion
print(y.value_counts())

tipo
queja           134
reclamo         127
peticion        112
felicitacion     99
Name: count, dtype: int64


In [46]:
# Separa entre sets de entramiento y testeo
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  #mantiene la distribucion
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


Train size: 377
Test size: 95


In [47]:
# 5. Vectorizer TF-IDF
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)

# Ajusta datos de entrenamiento
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF train shape:", X_train_tfidf.shape)
print("TF-IDF test shape:", X_test_tfidf.shape)

# LogisticRegression
clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    multi_class="auto"
)

clf.fit(X_train_tfidf, y_train)

print("Modelo entrenado.")

TF-IDF train shape: (377, 374)
TF-IDF test shape: (95, 374)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Modelo entrenado.


In [48]:
#Predicciones
y_pred = clf.predict(X_test_tfidf)


acc = accuracy_score(y_test, y_pred)
print(f"Accuracy en test: {acc:.4f}")

# Reporte clasificacion con precisión, recall, F1 por clase)
print("\nReporte de clasificacion:")
print(classification_report(y_test, y_pred))

# Matriz de confusion
labels = sorted(y.unique())
cm = confusion_matrix(y_test, y_pred, labels=labels)

cm_df = pd.DataFrame(cm, index=[f"true_{l}" for l in labels],
                        columns=[f"pred_{l}" for l in labels])

print("\nConfusion matrix:")
cm_df


Accuracy en test: 1.0000

Reporte de clasificacion:
              precision    recall  f1-score   support

felicitacion       1.00      1.00      1.00        20
    peticion       1.00      1.00      1.00        22
       queja       1.00      1.00      1.00        27
     reclamo       1.00      1.00      1.00        26

    accuracy                           1.00        95
   macro avg       1.00      1.00      1.00        95
weighted avg       1.00      1.00      1.00        95


Confusion matrix:


,pred_felicitacion,pred_peticion,pred_queja,pred_reclamo
true_felicitacion,20,0,0,0
true_peticion,0,22,0,0
true_queja,0,0,27,0
true_reclamo,0,0,0,26


In [52]:
import joblib

# 10. guadra el TF-IDF y el modelo
joblib.dump(tfidf, "tfidf_vectorizer.joblib")
joblib.dump(clf, "modelo_logreg_solicitudes.joblib")

print("Modelo guardado.")


def predecir_tipo(texto: str) -> str:
    """
    Recibe un texto y devuelve el tipo de solicitud.
    """

    t = str(texto).lower()
    vec = tfidf.transform([t])
    pred = clf.predict(vec)[0]
    return pred

# Prueba
ejemplo = "El ciudadano agradece la atencion"
print("Texto de prueba:", ejemplo)
print("Predicción:", predecir_tipo(ejemplo))

Modelo guardado.
Texto de prueba: El ciudadano agradece la atencion
Predicción: felicitacion
